# Классификация

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Библиотеки

In [ ]:
import os
#from google.colab import drive
from tqdm import tqdm
import pickle

import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Загружаем файлы

Проверям папку

In [ ]:
comments_dir = "/content/drive/My Drive/weights"
os.listdir(comments_dir)[:5]

['df_test.pickle', 'df_train.pickle']

In [ ]:
%%time

with open(os.path.join(comments_dir, 'df_train.pickle'), mode='rb') as pandas_file:
    train_df = pickle.load(pandas_file)
    
with open(os.path.join(comments_dir, 'df_test.pickle'), mode='rb') as pandas_file:
    test_df = pickle.load(pandas_file)

Wall time: 687 ms


In [ ]:
train_df.sample(2)

,author,body,created_utc,link_id,parent_id,score,who,embedding
9979,HotKreemy,I thought it was something about children with...,1604461055,t3_jl8d39,t1_gaogtfl,1,0.0,"[-0.25099966, -0.48429942, 0.33789465, 0.40046..."
67472,bro8619,My only issue with that is the trend here is d...,1601378973,t3_j1pmy5,t1_g7181lf,1,1.0,"[0.284637, 0.6118825, 0.6989911, -0.6090106, 0..."


In [ ]:
test_df.sample(2)

,author,body,created_utc,link_id,parent_id,score,who,embedding
13811,webdes03,Or they will cost Biden the election by not vo...,1601390815,t3_j1z2g6,t1_g722avz,1,1.0,"[0.5652691, 0.13292174, 0.74996924, 0.697091, ..."
2749,Kek_priest_Wunderbar,"Bruh, too early for me to read that Spanish. C...",1581080517,t3_eztav8,t1_fgpol80,1,0.0,"[0.26029673, 0.67823195, 0.87307334, -1.15178,..."


# Классификатор

Достаем признаки и таргет

In [ ]:
train_features = []
for i in range(len(train_df['embedding'])):
    train_features.append(train_df['embedding'].iloc[i])
    
test_features = []
for i in range(len(test_df['embedding'])):
    test_features.append(test_df['embedding'].iloc[i])

train_x = pd.DataFrame(np.array(train_features))
test_x = pd.DataFrame(np.array(test_features))

train_y = train_df['who']
test_y = test_df['who']

In [ ]:
print('Размер признаков трейна', train_x.shape)
print('Размер признаков теста', test_x.shape)

print('Размер таргет трейна', train_y.shape)
print('Размер таргет теста', test_y.shape)

Размер признаков трейна (100000, 1024)
Размер признаков теста (20000, 1024)
Размер таргет трейна (100000,)
Размер таргет теста (20000,)


Обучим модель и посмотрим на результат

In [ ]:
logreg = LogisticRegression(n_jobs=1, C=1e5, random_state=333, 
#                            class_weight='balanced', solver='liblinear',
                           )
logreg.fit(train_x, train_y)
y_pred = logreg.predict(test_x)

print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       0.70      0.72      0.71     10000
         1.0       0.71      0.69      0.70     10000

    accuracy                           0.70     20000
   macro avg       0.70      0.70      0.70     20000
weighted avg       0.70      0.70      0.70     20000



C:\Users\Home\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
